In [1]:
from sklearn.datasets import fetch_20newsgroups
import nltk.data
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import logging
from gensim.models import Word2Vec,KeyedVectors
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from nltk.stem.porter import PorterStemmer
from stemming.porter2 import stem
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Imputer

In [2]:
#loading the news dataset from sklearn dataset library
twenty_train = fetch_20newsgroups(subset='train',shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

# tokenize to sentences based on the notations followed in english literature
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
print("\n".join(twenty_train.data[0].split("\n")[:3]))
print(twenty_train.target_names[twenty_train.target[0]])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
rec.autos


In [3]:
# function to parse sentences to words and remove stopwords from sentence
def sentence_to_wordlist( sentence, remove_stopwords=True ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    text = BeautifulSoup(sentence).get_text()
    #  
    # 2. Remove non-letters
    text = re.sub("[^a-zA-Z]"," ",text)
    #
    # 3. Convert words to lower case and split them
    words = text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [4]:
# function to parse text to sentences using tokenizer mentioned above
def text_to_sentences(text,tokenizer,remove_stopwords=True):
    raw_sentences = tokenizer.tokenize(text.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( sentence_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [5]:
#train_sentences and test_sentences contain the filtered sentences of training and testing data respectively

train_sentences = []  # Initialize an empty list of sentences
test_sentences = []

print "Parsing sentences from training set"
for i in range(len(twenty_train.data)):
    train_sentences += [[stem(str(word)) for word in sentence] \
                        for sentence in [text_to_sentences(twenty_train.data[i], tokenizer)]]
print len(train_sentences)
print train_sentences[0]

print "Parsing sentences from unlabeled set"
for i in range(len(twenty_test.data)):
    test_sentences += [[stem(str(word)) for word in sentence] \
                       for sentence in [text_to_sentences(twenty_test.data[i], tokenizer)]]
print len(test_sentences)
print test_sentences[0]

Parsing sentences from training set


/home/dell/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/dell/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/dell/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
/home/dell/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: ".." looks like a filename, not markup. You shouldpro

11314
["[u'lerxst', u'wam', u'umd', u'edu', u'thing', u'subject', u'car']", "[u'nntp', u'posting', u'host', u'rac', u'wam', u'umd', u'edu', u'organization', u'university', u'maryland', u'college', u'park', u'lines', u'wondering', u'anyone', u'could', u'enlighten', u'car', u'saw', u'day']", "[u'door', u'sports', u'car', u'looked', u'late', u'early']", "[u'called', u'bricklin']", "[u'doors', u'really', u'small']", "[u'addition', u'front', u'bumper', u'separate', u'rest', u'body']", "[u'know']", "[u'anyone', u'tellme', u'model', u'name', u'engine', u'specs', u'years', u'production', u'car', u'made', u'history', u'whatever', u'info', u'funky', u'looking', u'car', u'please', u'e', u'mail']", "[u'thanks', u'il', u'brought', u'neighborhood', u'lerxst']"]
Parsing sentences from unlabeled set


/home/dell/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "/" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


7532
["[u'v', u'mb', u'k', u'ubvmsd', u'cc', u'buffalo', u'edu', u'neil', u'b', u'gandler', u'subject', u'need', u'info', u'bonneville', u'organization', u'university', u'buffalo', u'lines', u'news', u'software', u'vax', u'vms', u'vnews', u'nntp', u'posting', u'host', u'ubvmsd', u'cc', u'buffalo', u'edu', u'little', u'confused', u'models', u'bonnevilles']", "[u'heard', u'le', u'se', u'lse', u'sse', u'ssei']", "[u'could', u'someone', u'tell', u'differences', u'far', u'features', u'performance']", "[u'also', u'curious', u'know', u'book', u'value', u'prefereably', u'model']", "[u'much', u'less', u'book', u'value', u'usually', u'get']", "[u'words', u'much', u'demand', u'time', u'year']", "[u'heard', u'mid', u'spring', u'early', u'summer', u'best', u'time', u'buy']", "[u'neil', u'gandler']"]


In [6]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
# Initialize and train the model (this will take some time)

print "Training model..."
model = Word2Vec(train_sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=False)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2017-04-11 18:14:10,705 : INFO : collecting all words and their counts
2017-04-11 18:14:10,706 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-04-11 18:14:10,877 : INFO : PROGRESS: at sentence #10000, processed 166860 words, keeping 130308 word types
2017-04-11 18:14:10,895 : INFO : collected 144031 word types from a corpus of 187884 raw words and 11314 sentences
2017-04-11 18:14:10,896 : INFO : Loading a fresh vocabulary


Training model...


2017-04-11 18:14:10,982 : INFO : min_count=40 retains 28 unique words (0% of original 144031, drops 144003)
2017-04-11 18:14:10,984 : INFO : min_count=40 leaves 7150 word corpus (3% of original 187884, drops 180734)
2017-04-11 18:14:10,985 : INFO : deleting the raw counts dictionary of 144031 items
2017-04-11 18:14:10,991 : INFO : sample=0.001 downsamples 28 most-common words
2017-04-11 18:14:10,993 : INFO : downsampling leaves estimated 1012 word corpus (14.2% of prior 7150)
2017-04-11 18:14:10,994 : INFO : estimated required memory for 28 words and 300 dimensions: 81200 bytes
2017-04-11 18:14:10,995 : INFO : resetting layer weights
2017-04-11 18:14:10,998 : INFO : training model with 4 workers on 28 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2017-04-11 18:14:11,000 : INFO : expecting 11314 sentences, matching count from corpus used for vocabulary survey
2017-04-11 18:14:11,238 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-

In [7]:
def makeFeatureVec(words,model,num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float64")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #sorted_set = set(model.sort_vocab())
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [8]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
    # Print a status message every 1000th review
        if counter%1000. == 0.:
            print "Review %d of %d" % (counter, len(reviews))

    # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
    # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

In [9]:
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. 

trainDataVecs = getAvgFeatureVecs( train_sentences, model, num_features )
testDataVecs = getAvgFeatureVecs( test_sentences, model, num_features )

Review 0 of 11314
Review 1000 of 11314
Review 2000 of 11314
Review 3000 of 11314
Review 4000 of 11314
Review 5000 of 11314
Review 6000 of 11314
Review 7000 of 11314
Review 8000 of 11314
Review 9000 of 11314
Review 10000 of 11314
Review 11000 of 11314
Review 0 of 7532


/home/dell/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: RuntimeWarning: invalid value encountered in divide


Review 1000 of 7532
Review 2000 of 7532
Review 3000 of 7532
Review 4000 of 7532
Review 5000 of 7532
Review 6000 of 7532
Review 7000 of 7532


In [10]:
knn = KNeighborsClassifier(n_neighbors=4)
trainDataVecs = Imputer().fit_transform(trainDataVecs)
testDataVecs = Imputer().fit_transform(testDataVecs)

In [13]:
#Sequential Floating Forward Selection
sffs = SFS(knn,
          k_features = 20,
          forward = True,
          floating = True,
          scoring = 'accuracy',
          cv = 4)
sffs = sffs.fit(trainDataVecs,twenty_train.target)

#print sfs1.subsets_
print sffs.k_feature_idx_
print sffs.k_score_

trainDataVecs_sfs = sffs.transform(trainDataVecs)
testDataVecs_sfs = sffs.transform(testDataVecs)

# Fit the estimator using the new feature subset
# and make a prediction on the test data
knn.fit(trainDataVecs_sfs, twenty_train.target)
y_pred = knn.predict(testDataVecs_sfs)

# Compute the accuracy of the prediction
acc = float((twenty_test.target == y_pred).sum()) / y_pred.shape[0]
print('Test set accuracy: %.2f %%' % (acc * 100))


STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

(34, 133, 73, 139, 238, 47, 144, 22, 153, 218, 286, 159)
0.0986346712853
Test set accuracy: 8.31 %


In [14]:
#Sequential Floating Backward Selection
sfbs = SFS(knn,
          k_features = 20,
          forward = False,
          floating = True,
          scoring = 'accuracy',
          cv = 4)
sfbs = sfbs.fit(trainDataVecs,twenty_train.target)

#print sfs1.subsets_
print sfbs.k_feature_idx_
print sfbs.k_score_

trainDataVecs_sbs = sfbs.transform(trainDataVecs)
testDataVecs_sbs = sfbs.transform(testDataVecs)

# Fit the estimator using the new feature subset
# and make a prediction on the test data
knn.fit(trainDataVecs_sbs, twenty_train.target)
y_pred = knn.predict(testDataVecs_sbs)

# Compute the accuracy of the prediction
acc = float((twenty_test.target == y_pred).sum()) / y_pred.shape[0]
print('Test set accuracy: %.2f %%' % (acc * 100))


STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,